<div align='center'>
<font size = 7><font face="Product-Sans"><b><font color= "4285F4">G</font><font color= "DB4437">o</font><font color = "F4B400">o</font><font color= "4285F4">g</font><font color= "0F9D58">l</font><font color= "DB4437">e</font></b></font> - <font color = "C99700">Notre Dame</font></a> <font color = "4285F4">XLS Playground</font></font>
</div>

<div align='center'>
<img src='https://google.github.io/xls/images/xls_logo.svg' alt='XLS Logo' width=400><img src='https://raw.githubusercontent.com/mmorri22/cse30342/main/ND%20Chip%20Logo.png' alt='ND Chip Logo' width=180>
<img src="https://opensource.google/static/images/os-anim-main.gif" width=180>
</div>

<div align='center'>
<font size = 6><font color = "00843D">Lecture 25 In-Class Starter - Advanced Combinational Logic</font></a></font>
</div>

## XLS Setup

For each new Colab notebook, you will need to run the XLS setup again. If your computer switches networks, or you restart, you will need to run those commands again. This consists of the same two setup steps from Chapter 1. You must run both in order to properly run the XLS flow.


> Note: Here is the common error message that will occur if you ran a DSLX cell and you need need to re-run the setup. If you encounter this message, simply re-run these two steps and the error will be resolved when you go back to that cell:
>
> <code>UsageError: Cell magic `%%dslx` not found.</code>

# <font color = "red">Design Run Setup</font>

## To set up the design flow, run the next cell to set up the back end of XLS

> You will need to run this setup at every new runtime instance.

In [ ]:
#@title Start-up Step 1: XLS and OpenRoad scripts {run:"auto"}

!rm -rf *

# Import required Python libraries
import os
import pathlib
import sys
import jinja2
import IPython.display
import PIL.Image
import graphviz
import pathlib

from IPython.display import display, display_png

# Set Stable XLS Version for classroom environment
xls_version = 'v0.0.0-4699-gfb023174' #@param {type:"string"}

!echo '📦 downloading xls-{xls_version}'
!curl --show-error -L https://github.com/proppy/xls/releases/download/{xls_version}/xls-{xls_version}-linux-x64.tar.gz | tar xzf - --strip-components=1
!echo '🧪 setting up colab integration'
!python -m pip install --quiet --no-cache-dir --ignore-installed https://github.com/proppy/xls/releases/download/{xls_version}/xls_colab-0.0.0-py3-none-any.whl
!python -m pip install logger
!python -m pip install colabtools
import logger
import xls.contrib.colab
_ = xls.contrib.colab.register_dslx_magic()

# Must verify xls_work_dir is created
!if test -d xls_work_dir; then echo "xls_work_dir exists"; else mkdir xls_work_dir;  fi

#@title  First Run Only #4 - OpenRoad Setup {run:"auto"}

yosys_version = '0.38_93_g84116c9a3' #@param {type:"string"}
openroad_version = '2.0_12381_g01bba3695' #@param {type:"string"}
rules_hdl_version = '2eb050e80a5c42ac3ffdb7e70392d86a6896dfc7' #@param {type:"string"}

# Install stable OpenROAD Version
!echo '🛣️ installing openroad and friends'
!curl -L -O https://repo.anaconda.com/miniconda/Miniconda3-py310_24.1.2-0-Linux-x86_64.sh
!bash Miniconda3-py310_24.1.2-0-Linux-x86_64.sh -b -p conda-env/
import pathlib
conda_prefix_path = pathlib.Path('conda-env')
CONDA_PREFIX = str(conda_prefix_path.resolve())
%env CONDA_PREFIX={CONDA_PREFIX}
!conda-env/bin/conda install -yq -c "litex-hub" openroad={openroad_version} yosys={yosys_version}

!python -m pip install gdstk tqdm

!gsutil cp gs://proppy-eda/pdk_info_asap7.zip .
!gsutil cp gs://proppy-eda/pdk_info_sky130.zip .

!unzip -q -o pdk_info_asap7.zip
!unzip -q -o pdk_info_sky130.zip

!echo '🧰 generating PDK metadata'
!curl --show-error -L  https://github.com/hdl/bazel_rules_hdl/archive/{rules_hdl_version}.tar.gz | tar xzf - --strip-components=1
!curl -L -O https://github.com/protocolbuffers/protobuf/releases/download/v24.3/protoc-24.3-linux-x86_64.zip
!unzip -q -o protoc-24.3-linux-x86_64.zip
!{sys.executable} -m pip install protobuf

!echo '📁 organizing PDK for XLS and OpenROAD Flows'
!wget https://raw.githubusercontent.com/mmorri22/cse30321/main/xls/xls_setup.py
!wget https://raw.githubusercontent.com/mmorri22/cse30321/main/xls/sky130_data_pdk_info.textproto
!python xls_setup.py
!mv /content/sky130_data_pdk_info.textproto /content/com_google_skywater_pdk_sky130_fd_sc_hd/sky130_data_pdk_info.textproto
!echo '🖼️ Setup for viewing 3D GDSII File'
!python -m pip install numpy
!python -m pip install gdspy
!python -m pip install numpy-stl
!python -m pip install triangle
!python -m pip install k3d

# gdspy is used to open the gds file
import gdspy

# Used to write the output stl file (Why we installed numpy-stl)
from stl import mesh

# Using numpy will permit fast calculations on lots of points
import numpy as np
import matplotlib

# Required to triangulate polygons
import triangle

# To render in 3d
import k3d

In [ ]:
#@title Start-up Step 2: Select your PDK {run:"auto"}

pdk = 'sky130' #@param ["asap7", "sky130"] {allow-input: false}

xls.contrib.colab.pdk = pdk


#@title Select your PDK {run:"auto"}

!bin/protoc --python_out=. pdk/proto/pdk_info.proto
!ln -sf pdk/proto/pdk_info_pb2.py
import pdk_info_pb2

import enum
import dataclasses
import json
import pathlib
import subprocess
from typing import Any, Callable, Dict, Optional, Union

from google.colab import widgets
from google.protobuf import text_format
import pandas as pd

yosys = conda_prefix_path / 'bin/yosys'
openroad = conda_prefix_path / 'bin/openroad'
yosys_tcl = 'synthesis/synth.tcl'

default_work_dir = xls.contrib.colab.default_work_dir

def pdk_info_proto(
    path: pathlib.Path, optional: bool = False
) -> Optional[pdk_info_pb2.PdkInfoProto]:
  """Load PDK info from prototext.

  Args:
    path: path to prototext file.
    optional: if True, failure to access the pdk info will not produce an error.

  Returns:
    Decoded pdk info proto or None if optional.
  """
  if optional and not path.exists():
    return None
  with path.open('r') as f:
    proto = pdk_info_pb2.PdkInfoProto()
    text_format.Parse(f.read(), proto)
    return proto

pdks = {

    'asap7': {
        'delay_model': 'asap7',
        'pdk_info': pdk_info_proto(
            pathlib.Path('asap7/asap7_data_pdk_info.textproto'),
        ),
    },

    'sky130': {
        'delay_model': 'sky130',
        'pdk_info': pdk_info_proto(
            pathlib.Path('com_google_skywater_pdk_sky130_fd_sc_hd/sky130_data_pdk_info.textproto'),
        ),
    },
}

@dataclasses.dataclass(frozen=True)
class RelativeCoreArea:
  utilization_percent: float


@dataclasses.dataclass(frozen=True)
class AbsoluteCoreArea:
  core_width_microns: int
  core_padding_microns: int


@enum.unique
class ImplementationStep(enum.Enum):
  """Steps in the implementation flow."""

  XLS = 'xls'
  SYNTHESIS = 'synthesis'
  PLACEMENT = 'placement'


class PdkRuntimeError(RuntimeError):
  pass


class OpenroadRuntimeError(RuntimeError):
  pass


class OpenstaRuntimeError(RuntimeError):
  pass


class YosysRuntimeError(RuntimeError):
  pass


@dataclasses.dataclass(frozen=True)
class SynthesisResults:
  synth_v: pathlib.Path
  design_stats: pd.DataFrame
  cell_stats: pd.DataFrame


def run_synthesis(
    *,
    selected_pdk: Optional[str] = None,
    work_dir: pathlib.Path = default_work_dir,
    silent: bool = False,
) -> SynthesisResults:
  """Run synthesis with Yosys.

  Args:
    selected_pdk: The pdk to use.
    work_dir: Directory that contains verilog and will be where outputs are put.
    silent: Suppress output.

  Returns:
    Metrics from running synthesis.

  Raises:
    PdkRuntimeError: on PDK error.
    YosysRuntimeError: on yosys error.
  """
  if selected_pdk is None:
    selected_pdk = pdk
  pdk_info = pdks[selected_pdk]['pdk_info']
  if pdk_info is None:
    raise PdkRuntimeError(f'PDK "{selected_pdk}" is restricted')

  liberty = (pathlib.Path(pdk) / pathlib.Path(pdk_info.liberty_path).name).resolve()
  synth_v = (work_dir / 'user_module_synth.v').resolve()
  synth_v_flist = (work_dir / 'user_module_synth_v.flist').resolve()
  synth_uhdm_flist = (work_dir / 'user_module_synth_uhdm.flist').resolve()
  synth_uhdm_flist.touch()
  synth_stats_json = (work_dir / 'user_module_synth_stats.json').resolve()
  dont_use_args = ' '.join(
      f'-dont_use {pat}'
      for pat in pdk_info.do_not_use_cell_list
  )
  # run yosys synthesis
  with synth_v_flist.open('w') as f:
    top_v = work_dir / 'user_module.sv'
    f.write(str(top_v.resolve()))
  !FLIST='{synth_v_flist}' ABC_SCRIPT='' CONSTR='' TOP='user_module' OUTPUT='{synth_v}' UHDM_FLIST='{synth_uhdm_flist}' LIBERTY='{liberty}' STATS_JSON='{synth_stats_json}' DONT_USE_ARGS='{dont_use_args}' {yosys} -c '{yosys_tcl}'
  with synth_stats_json.open('r') as f:
    synth_stats = json.load(f)
  design_stats = synth_stats['design']
  cells_stats = design_stats.pop('num_cells_by_type')
  design_stats = pd.DataFrame.from_dict(
      design_stats, orient='index', columns=['cells']
  )
  cells_stats = pd.DataFrame.from_dict(
      cells_stats, orient='index', columns=['stats']
  )

  return SynthesisResults(
      synth_v=synth_v, design_stats=design_stats, cell_stats=cells_stats
  )


def run_opensta(
    *,
    selected_pdk: Optional[str] = None,
    work_dir: pathlib.Path = default_work_dir,
    silent: bool = False,
) -> pd.DataFrame:
  """Run OpenSta and collect timing metrics.

  Args:
    selected_pdk: The pdk to use.
    work_dir: Directory that contains verilog.
    silent: Suppress output.

  Returns:
    Dataframe containing timing report.

  Raises:
    OpenstaRuntimeError: on OpenSTA error.
    PdkRuntimeError: on PDK error.
  """
  if selected_pdk is None:
    selected_pdk = pdk
  pdk_info = pdks[selected_pdk]['pdk_info']
  if pdk_info is None:
    raise PdkRuntimeError(f'PDK "{selected_pdk}" is restricted')

  liberty = pathlib.Path(pdk) / pdk_info.liberty_path
  tech_lef = pathlib.Path(pdk) / pdk_info.tech_lef_path
  read_cell_lefs = '\n'.join(
      f'read_lef {pathlib.Path(pdk) / cell_lef_path}'
      for cell_lef_path in pdk_info.cell_lef_paths
  )
  synth_v = work_dir / 'user_module_synth.v'
  top = 'user_module'
  opensta_log = work_dir / 'user_module_sta.log'

  openroad_script = f"""
  sta::redirect_file_begin {opensta_log}
  read_lef {tech_lef}
  {read_cell_lefs}
  read_liberty {liberty}
  read_verilog {synth_v}
  link_design  {top}
  report_checks -unconstrained
  sta::redirect_file_end
  """
  openroad_tcl = work_dir / 'openroad_sta.tcl'
  with openroad_tcl.open('w') as f:
    f.write(openroad_script)

  # run opensta static timing analysis
  !{openroad} {openroad_tcl} -exit

  columns = ['delay', 'time', 'edge', 'net', 'gate']

  import re
  def sta_report_paths(opensta_log):
    with open(opensta_log) as f:
      sta_report = f.read()
    m = re.search(r'---+(.*)---+', sta_report, flags=re.M | re.S)
    for path in m.group(1).split('\n')[1:-2]:
      parts = path.split(None, maxsplit=len(columns) - 1)
      yield float(parts[0]), float(parts[1]), parts[2], parts[3], parts[4]

  df = pd.DataFrame.from_records(sta_report_paths(opensta_log), columns=columns)
  df['gate'] = df['gate'].str.replace('[()]', '', regex=True)

  return df


@dataclasses.dataclass(frozen=True)
class PlacementResults:
  openroad_global_placement_layout: pathlib.Path
  area: pd.DataFrame
  metrics: pd.DataFrame
  power: pd.DataFrame


def run_placement(
    *,
    clock_period_ps: int,
    placement_density: float,
    core_area: Union[RelativeCoreArea, AbsoluteCoreArea],
    selected_pdk: Optional[str] = None,
    work_dir: pathlib.Path = default_work_dir,
    silent: bool = False,
) -> PlacementResults:
  """Run OpenRoad placement.

  Args:
    clock_period_ps: Clock period in picoseconds.
    placement_density: Placement density in [0.0, 1.0].
    core_area: Relative or absolute core area specification.
    selected_pdk: The pdk to use.
    work_dir: Directory that contains verilog and will be where outputs are put.
    silent: Suppress output.

  Returns:
    Outputs from running placement.

  Raises:
    OpenroadRuntimeError: on OpenRoad error.
    OpenstaRuntimeError: on OpenSTA error.
    PdkRuntimeError: on PDK error.
    ValueError: on invalid inputs.
    YosysRuntimeError: on yosys error.
  """
  clock_period_ns = clock_period_ps / 1000.0
  if selected_pdk is None:
    selected_pdk = pdk
  pdk_info = pdks[selected_pdk]['pdk_info']
  if pdk_info is None:
    raise PdkRuntimeError(f'PDK "{selected_pdk}" is restricted')

  liberty = pathlib.Path(pdk) / pdk_info.liberty_path
  tech_lef = pathlib.Path(pdk) / pdk_info.tech_lef_path
  read_cell_lefs = '\n'.join(
      f'read_lef {pathlib.Path(pdk) / cell_lef_path}'
      for cell_lef_path in pdk_info.cell_lef_paths
  )

  if isinstance(core_area, AbsoluteCoreArea):
    die_side_microns = (
        core_area.core_width_microns + core_area.core_padding_microns * 2
    )
    core_side_microns = (
        core_area.core_width_microns + core_area.core_padding_microns
    )
    initialize_floorplan_args = (
        f' -die_area "0 0 {die_side_microns} {die_side_microns}" -core_area'
        f' "{core_area.core_padding_microns} {core_area.core_padding_microns} {core_side_microns} {core_side_microns}"'
    )
  elif isinstance(core_area, RelativeCoreArea):
    initialize_floorplan_args = (
        f' -utilization {core_area.utilization_percent} -aspect_ratio 1.0'
    )
  else:
    raise ValueError(
        'Expected core_area to be AbsoluteCoreArea or RelativeCoreArea, got'
        f' {core_area!r}'
    )

  initialize_floorplan_command = (
      f'initialize_floorplan -site "{pdk_info.cell_site}"'
      f' {initialize_floorplan_args}'
  )

  def source_pdk_info_tcl(path):
    return f'source {pathlib.Path(pdk) / path}' if path else ''

  source_tracks_file = source_pdk_info_tcl(pdk_info.tracks_file_path)
  source_rc_script_configuration = source_pdk_info_tcl(
      pdk_info.rc_script_configuration_path
  )
  source_pdn_config = source_pdk_info_tcl(pdk_info.pdn_config_path)
  if pdk_info.tapcell_tcl_path:
    tapcell_command = source_pdk_info_tcl(pdk_info.tapcell_tcl_path)
  else:
    tapcell_command = (
        f'tapcell -distance {pdk_info.tapcell_distance} -tapcell_master'
        f' {pdk_info.tap_cell}'
    )

  synth_v = work_dir / 'user_module_synth.v'
  openroad_metrics = work_dir / 'openroad_metrics.json'
  openroad_global_placement_layout = work_dir / 'openroad_global_placement.png'

  openroad_script = f"""
  read_lef {tech_lef}
  {read_cell_lefs}
  read_liberty {liberty}
  read_verilog {synth_v}
  link_design user_module
  {initialize_floorplan_command}
  {source_tracks_file}
  insert_tiecells {pdk_info.tie_high_port} -prefix "TIE_ONE_"
  insert_tiecells {pdk_info.tie_low_port} -prefix "TIE_ZERO_"
  create_clock [get_ports clk] -period {clock_period_ns}
  {source_rc_script_configuration}
  set_wire_rc -signal -layer "{pdk_info.wire_rc_signal_metal_layer}"
  set_wire_rc -clock  -layer "{pdk_info.wire_rc_clock_metal_layer}"
  place_pins -hor_layers {pdk_info.pin_horizontal_metal_layer} -ver_layers {pdk_info.pin_vertical_metal_layer}
  {tapcell_command}
  {source_pdn_config}
  pdngen -verbose
  global_placement -timing_driven -routability_driven -density {placement_density} -pad_left {pdk_info.global_placement_cell_pad} -pad_right {pdk_info.global_placement_cell_pad}
  remove_buffers
  estimate_parasitics -placement
  repair_design
  repair_timing
  utl::metric "utilization_percent" [rsz::utilization]
  utl::metric "design_area" [rsz::design_area]
  utl::metric "power" [sta::design_power [sta::parse_corner {{}}]]
  utl::metric "wns" [sta::worst_slack -max]
  report_power
  report_design_area
  if {{[info procs save_image] == "save_image"}} {{
    save_image -resolution 0.005 "{openroad_global_placement_layout}"
  }}
  """
  openroad_tcl = work_dir / 'place.tcl'
  with openroad_tcl.open('w') as f:
    f.write(openroad_script)
  !QT_QPA_PLATFORM=minimal {openroad} -metrics {openroad_metrics} -exit {openroad_tcl}

  with open(work_dir / 'openroad_metrics.json', 'r') as f:
    metrics = json.loads(f.read())
  df_area = pd.DataFrame.from_dict(
      {
          'global placement': [
              float(metrics['design_area']) * 1e12,
              float(metrics['utilization_percent']) * 100,
          ]
      },
      columns=['area', 'utilization'],
      orient='index',
  )
  metrics_power = [float(m) * 1e6 for m in metrics['power'].split(' ')]
  df_power = pd.DataFrame().from_dict(
      {
          'sequential': metrics_power[4:8],
          'combinational': metrics_power[8:12],
          'clock': metrics_power[12:16],
          'macro': metrics_power[16:20],
          'pad': metrics_power[20:],
          'total': metrics_power[0:4],
      },
      orient='index',
      columns=['internal', 'switching', 'leakage', 'total'],
  )
  df_metrics = (
      pd.DataFrame.from_records([metrics])
      .transpose()
      .set_axis(['metrics'], axis=1)
  )
  return PlacementResults(
      openroad_global_placement_layout=openroad_global_placement_layout,
      area=df_area,
      metrics=df_metrics,
      power=df_power,
  )

## In-Class Reading Review - Lecture 25

> Before we start, be sure to run the first two steps to initialize the XLS and OpenRoad script so you mau continue your coding.

Discuss for 3-minutes, then class discussion:
<ul>
  <li>Define <b>Hardware Description Languages</b></li>
  <li>Describe <b>domain-specific language</b></li>
  <li>Describe <b>Intermediate representation</b></li>
  <li>Describe <b>Process Development Kit</b></li>
  <li>Describe <b>Separation of Concerns</b></li>
  <li>Describe <b>Standard Cells</b></li>
</ul>

## Going a little deeper - Data Forwarding Comparison

Recall that in the RISC-V Architecture, we used a <b>Hazard Detection Unit</b> as a control unit to determine if one of the pipelined register hazard conditions exists in the datapath. The hazard detection logic was:

<code>EX/MEM.RegisterRd = ID/EX.RegisterRs1</code><br>
<code>EX/MEM.RegisterRd = ID/EX.RegisterRs2</code><br>
<code>MEM/WB.RegisterRd = ID/EX.RegisterRs1</code><br>
<code>MEM/WB.RegisterRd = ID/EX.RegisterRs2</code><br>

To this point, we have only discussed DSLX methods with return values with a direct correlation of input to output. But you learned that carry adders use <b>intermediate</b> values in order to perform these operations at the logic level. So how do we do that?

In [ ]:
#@title Simple Data-Forwarding Comparison logic {run:"auto"}

%%dslx --top=data_forwarding_logic --pipeline_stages=1 --flop_inputs=false --flop_outputs=false

// data_forwarding_logic will give us the selection between reg_exe, reg_dm, and reg_dm
// We will return a 2-bit signal to send to the ALU controllers


// Now, we will practice again with the data_forwarding_logic_test
#[test]
fn data_forwarding_logic_test() {

  // No Hazard
  let reg_rs1:u5 = u5:16;
  let reg_mem:u5 = u5:22;
  let reg_wb:u5  = u5:20;

  // Compare and see if we get rs1


  // Change reg_mem to 16 to create a hazard with rs1


  // Reset reg_mem and Change reg_write to 16 create a hazard with rs1


  // Now, let's see reg_mem and reg_wb to 16. Should return 3 since reg_write is ahead in the datapath


}


## Data Forarding Part 2 - Forwarding the 32-bit values back to the ALU

We haven't covered the ALU yet, but we can use these XLS techniques to implement multiplexers for the data forwarding inputs to the ALU. Recall from Lecture 12 our data forwarding logic:

<img src = "https://github.com/mmorri22/cse30321/blob/main/xls/lec24/forwarding%20logic.jpg?raw=true" width=600>

In [ ]:
#@title The 3-input 5-bit MUX for the Data Forwarding Unit {run:"auto"}

%%dslx --top=data_forwarding_mux --pipeline_stages=1 --flop_inputs=false --flop_outputs=false

// Now we will write the data_forwarding_mux



// Data Forwarding Mux Test
#[test]
fn data_forwarding_mux_test() {

  // No Hazard
  let data_rs1:u32 = u32:10;
  let data_mem:u32 = u32:25;
  let data_wb:u32 = u32:20;

  // No HZD. Use data_rs1
  assert_eq(data_forwarding_mux( data_rs1, data_mem, data_wb, u2:0 ), ( data_rs1 ) );

  // HZD says use data_mem
  assert_eq(data_forwarding_mux( data_rs1, data_mem, data_wb, u2:2 ), ( data_mem ) );

  // HZD says use data_wb
  assert_eq(data_forwarding_mux( data_rs1, data_mem, data_wb, u2:3 ), ( data_wb ) );

}

## Putting it all together: Simulating the Hazard Detection Process

Here, we will create an XLS module called <code>data_forwarding_example</code> where we simulate the entire hazard detection and data forwarding process.

In [ ]:
#@title The 3-input 5-bit MUX for the Data Forwarding Unit {run:"auto"}

%%dslx --top=data_forwarding_example --pipeline_stages=1 --flop_inputs=false --flop_outputs=false

fn data_forwarding_logic( reg_exe: u5, reg_dm: u5, reg_write: u5 ) -> u2 {

    match(reg_exe){
      reg_write => u2:3,
      reg_dm => u2:2,
      _ => u2:0
    }
}


fn data_forwarding_mux( data_exe: u32, data_mem: u32, data_wb: u32, sel: u2 ) -> u32 {

  // Put wb before reg so that if all 3 are the same, we select wb over reg
  match(sel){
      u2:3 => data_wb,
      u2:2 => data_mem,
      _ => data_exe
  }

}

// Now we will write a data_forwarding_example with all the appropriate inputs


#[test]
fn data_forwarding_test() {

  // No Hazard
  let reg_rs1:u5 = u5:16;
  let data_rs1:u32 = u32:10;
  let reg_rs2:u5 = u5:21;
  let data_rs2:u32 = u32:15;
  let reg_mem:u5 = u5:22;
  let data_mem:u32 = u32:25;
  let reg_wb:u5  = u5:20;
  let data_wb:u32 = u32:20;

  // Test the current elements to get data_rs1 and data_rs2

  // Change reg_mem to 16 to create a hazard with rs1

  // Reset reg_mem to 28 Change reg_write to 16 create a hazard with rs1

  // Now, let's see reg_mem and reg_wb to 16, and then ensure it gives us data_wb and not data_reg

  // Now we will do all the same tests with rs2
  // Change reg_mem to 16 to create a hazard with rs1

  // Change reg_write to 21 create a hazard with rs1

  // Now, let's see reg_mem and reg_wb to 21, and then ensure it gives us data_wb and not data_reg
}

## Immediate Generate Logic in RISC-V and array manipulation

Recall that, in Lectures 04 and 05, we discussed the process of developing the immediate generate value. Specifically, that there was a difference between the logic of the original immediate generate in <a href = "https://www2.eecs.berkeley.edu/Pubs/TechRpts/2011/EECS-2011-62.pdf">original RISC-V specification</a> in 2011. Here was the original specification for those instructions:

<img src = "https://github.com/mmorri22/cse30321/blob/main/xls/lec24/original_immediate.png?raw=true" width=600>

We also learned this specification was considerably different than the <a href = "https://www2.eecs.berkeley.edu/Pubs/TechRpts/2016/EECS-2016-1.pdf">updated specification</a> in 2017.

<img src = "https://github.com/mmorri22/cse30321/blob/main/xls/lec24/updated_immediate.png?raw=true" width=600>

This re-design inevitably causes confusion and consternation among students. The original design appears cleaner and simpler, whereas the new design looks complex and challenging. To address student concerns, in lecture I presented their specification and made the following claim:
> "This design reduces hardware costs for low-end implementations that re-use the ALU datapath to compute branch targets.

And most students go, sure, whatever you say... But let's <i>not</i> take Professor Morrison's word for it, and let's <i>not</i> take the inventors of RISC-V's word for it.

<b>Let's prove it using the XLS synthesis tool!</b>

### Array Accessing

We can set intermediate values in DSLX by accessing the range of elements. We note here that the notation uses the same notation as <code>range</code> in Python in that the last value is *exclusive*. For example, <code>[0:8]</code> actually gets the 8 bits from 0 to 7.

Consider the following DSLX code:

  <code>let opcode:u7 = instr[0:7];</code>

This code performs the following:
<ul>
  <li><code>let opcode:u7</code> - Creates a 7-bit unsigned intermediate</li>
  <li><code>instr[0:7];</code> - Sets opcode equal to bits 0-6</li>
</ul>

## Building the 2011 Specification Table

Next, we need to recall our <a href = "https://github.com/mmorri22/cse30321/blob/main/documents/Minimized%20Green%20Sheet%20for%20Exam%201%20Questions.pdf">reduced RISC-V Green Sheet</a> and correlate the opcode values to the immediate results. For this task, we will stick with 32-bit instructions.

|Opcode|Integer Value|Instruction|Action|
|:--|:--|:--|:--|
|0110011|51|all R-Type|No Immediate to Generate - Put all 0's for 32 bits|
|0000011|3|lw - I-type|Sign Extend to 32 bits 21 to 10|
|1101011|107|jalr - I-type|Sign Extend to 32 bits 21 to 10|
|0010011|19|All other I-type|Sign Extend to 32 bits 21 to 10|
|0100011|35|sw - B-type|Sign Extend to 32 bits 31 to 27 and 16 to 10|
|1100011|99|Branching - B-Type|Sign Extend to 32 bits 31 to 27 and 16 to 10|
|0110111|55|lui - L-type|Sign Extend to 32 bits 26 to 7|
|1101111|111|jal - J-Type|Shift Left by 1 (added to PC later) and Sign extend|


In [ ]:
%%dslx --top=risc_v_imm_gen_2011 --pipeline_stages=1 --flop_inputs=false --flop_outputs=false

fn risc_v_imm_gen_2011(instr: u32) -> u32{

  // Set the opcode
  let opcode:u7 = instr[0:7];

  // First, perform the simple R-Type case where we return 0
  if( opcode == u7:51 ){

      // Return the 32-bit value of 0
      u32:0
  }

  // All I-Type Instructions
  else if( opcode == u7:3 || opcode == u7:107 || opcode == u7:19 ){

      // Extract the immediate - recall 22 is *exclusive*
      let immediate:u12 = instr[10:22];

      // Check the sign bit at bit 21
      let sign_bit:u1 = instr[21+:u1];

      // Use signex to extend by 0 if positive
      match(sign_bit){
          u1:0 => signex(immediate, u32:0),
          _ => signex(immediate, u32:1)
      }
  }

  // All B-Type Instructions
  else if( opcode == u7:35 || opcode == u7:99 ){

      // Extract the immediate - 31 to 27 and 16 to 10
      let immediate:u12 = instr[27:] ++ instr[10:17];

      // Check the sign bit at bit 21
      let sign_bit:u1 = instr[31+:u1];

      // Use signex to extend by 0 if positive
      match(sign_bit){
          u1:0 => signex(immediate, u32:0),
          _ => signex(immediate, u32:1)
      }

  }

  // All J-Type Instructions
  else if( opcode == u7:111 ){

      // Extract the jump offset and shift left by 1
      let jump_offset:u25 = instr[7:31] ++ u1:0;

      // Check the sign bit at bit 31
      let sign_bit:u1 = instr[31+:u1];

      // Use signex to extend by 0 if positive
      match(sign_bit){
          u1:0 => signex(jump_offset, u32:0),
          _ => signex(jump_offset, u32:1)
      }

  }

  // lui Instruction - L-type
  else if( opcode == u7:55 ){

      // Extract the lui
      instr[7:27] ++ u12:0

  }

  // Finally, for any other case not covered, return 0 for the else case
  else{
      // Return the 32-bit value of 0
      u32:0
  }

}

#[test]
fn risc_v_imm_gen_2011_test_lw() {

  // Opcode for lw = 0000011
  // funct3 = 010
  // immediate = 32 = 000000100000
  // rs1 = 21 = 10101
  // rs2 = 22 = 10110
  let instr:u32 = u32:0b10110101010000001000000100000011;

  assert_eq(risc_v_imm_gen_2011(instr), u32: 32 );

}

#[test]
fn risc_v_imm_gen_2011_test_R_Type() {

  // Opcode for add = 0110011
  // funct10 = 0000000000
  // rs2 = 22 = 10110
  // rs1 = 21 = 10101
  // rd = 22 = 10110
  let instr:u32 = u32:0b10110101011011000000000000110011;

  assert_eq(risc_v_imm_gen_2011(instr), u32: 0 );

}

#[test]
fn risc_v_imm_gen_2011_test_B_Type() {

  // Opcode for sw = 0100011
  // funct3 = 010
  // imm[6:0] = 0011100
  // rs2 = 21 = 10101
  // rs1 = 22 = 10110
  // imm[11:7] = 00000
  let instr:u32 = u32:0b00000101101010100111000100100011;

  assert_eq(risc_v_imm_gen_2011(instr), u32: 28 );

}

#[test]
fn risc_v_imm_gen_2011_test_J_Type() {

  // Opcode for jal = 1101111
  // jump offset is 120/2 = 000000000000000000111100;
  let instr:u32 = u32:0b0000000000000000001111001101111;

  assert_eq(risc_v_imm_gen_2011(instr), u32: 120 );

}

#[test]
fn risc_v_imm_gen_2011_test_L_Type() {

  // Opcode for lui = 0110111
  // lui = 248 = 00000000000011111000;
  // rd = 18 = 10010
  let instr:u32 = u32:0b10010000000000000111110000110111;

  // Expected Result: 0b00000000000011111000000000000000
  assert_eq(risc_v_imm_gen_2011(instr), u32: 0b00000000000011111000000000000000 );

}

In [ ]:
#@title Run this cell to synthesize the risc_v_imm_gen_2011 cell {display-mode: "form"}
#@markdown - Click the ▷ button to run synthesis, static timing analysis and global placement

placement_density = 1 #@param {type:"slider", min:0, max:1.0, step:0.01}
clock_period_ps = 10000 #@param {type:"slider", min:0, max:100000, step:1}
clock_period_ns = clock_period_ps / 1000.0
core_area = 'absolute' # @param ["relative", "absolute"]

# @markdown ### core_area_relative
# @markdown compute core area from the design size
utilization_percent = 100 #@param {type:"slider", min:0, max:100, step:1}
# @markdown ### core_area_absolute
# @markdown set core area explicitly
core_width_microns = 50 #@param {type:"slider", min:0, max:1000, step:1}
core_padding_microns = 0 #@param {type:"slider", min:0, max:100, step:1}

from IPython.display import display, display_png
import IPython.display
import PIL.Image

if core_area == 'relative':
  core_area_value = RelativeCoreArea(utilization_percent)
else:
  core_area_value = AbsoluteCoreArea(core_width_microns, core_padding_microns)

tb = widgets.TabBar(['synthesis', 'netlist', 'timing', 'placement', 'area', 'power'])

# run yosys synthesis
with tb.output_to('synthesis', select=True):
  synth_results = run_synthesis()
  tb.clear_tab()

with tb.output_to('synthesis', select=False):
  grid = widgets.Grid(1, 2, header_row=False, header_column=False)
  with grid.output_to(0, 0):
    display(synth_results.cell_stats)
  with grid.output_to(0, 1):
    display(synth_results.design_stats)

# display gate level netlist
with tb.output_to('netlist', select=False):
  with synth_results.synth_v.open('r') as f:
    print(f.read())


# run opensta static timing analysis
with tb.output_to('timing', select=True):
  opensta_results = run_opensta()
  tb.clear_tab()

# display opensta report
with tb.output_to('timing', select=False):
  display(
      opensta_results.style.hide(axis='index')
      .background_gradient(subset=['delay'], cmap='Oranges')
      .bar(subset=['time'], color='lightblue')
  )

# run openroad placement
with tb.output_to('placement', select=True):
  placement_results = run_placement(
      clock_period_ps=clock_period_ps,
      placement_density=placement_density,
      core_area=core_area_value,
  )
  tb.clear_tab()

# display global placement layout
with tb.output_to('placement', select=False):
  if placement_results.openroad_global_placement_layout.exists():
    img = PIL.Image.open(placement_results.openroad_global_placement_layout)
    img = img.resize((500, 500))
    display_png(img)

# display area estimate
with tb.output_to('area', select=False):
  display(
      placement_results.area.style.format('{:.3f} μm²', subset=['area'])
      .format('{:.2f} %', subset=['utilization'])
      .bar(subset=['utilization'], color='lightblue', vmin=0, vmax=100)
  )

# display power metrics
with tb.output_to('power', select=False):
  display(
      placement_results.power.style.format('{:.3f} uW')
      .background_gradient(
          subset=pd.IndexSlice[
              placement_results.power.index[:-1], ['internal', 'switching', 'leakage']
          ],
          cmap='Oranges',
          axis=None,
      )
      .bar(subset=['total'], color='lightcoral')
      .bar(
          subset=pd.IndexSlice[placement_results.power.index[-1:], :],
          color='lightcoral',
          axis='columns',
      )
  )

### Updated RISC-V 2017 Immediate Generate Specification

|Opcode|Integer Value|Instruction|Action|
|:--|:--|:--|:--|
|0110011|51|all R-Type|No Immediate to Generate - Put all 0's for 16 bits|
|0000011|3|lw - I-type|Sign Extend to 32 bits 31 to 20|
|1101111|107|jalr - I-type|Sign Extend to 32 bits: 31, 19 to 12, 20, and 30 to 21|
|0010011|19|All other I-type|Sign Extend to 32 bits 31 to 20|
|0100011|35|sw - SB-type|Sign Extend to 32 bits: 31, 11, 30:25, 11 to 6, and then a 0 bit|
|1100011|99|Branching - SB-Type|Sign Extend to 32 bits: 31, 11, 30:25, 11 to 6, and then a 0 bit|
|0110111|55|lui - U-type|Sign Extend to 32 bits: 31 to 12|
|1101111|111|jal - UJ-Type|Shift Left by 1 - 31, 19:12, 20, 30 to 21 - (added to PC later) and Sign extend|

In [ ]:
%%dslx --top=risc_v_imm_gen_2017 --pipeline_stages=1 --flop_inputs=false --flop_outputs=false

fn risc_v_imm_gen_2017(instr: u32) -> u32{

  // Set the opcode
  let opcode:u7 = instr[0:7];

  // First, perform the simple
  if( opcode == u7:51 ){

      // Return the 32-bit value of 0
      u32:0
  }

  // All I-Type Instructions - We will practice I-Type Instructions case


  // All SB-Type Instructions


  // All UJ-Type Instructions


  // lui Instruction - U-type


  else{
      // Return the 32-bit value of 0
      u32:0
  }

}

#[test]
fn risc_v_imm_gen_2017_test_I_Type() {

  // Opcode for addi = 0010011
  // rd = 22 = 10110
  // funct3 = 010
  // rs1 = 21 = 10101
  // immediate = 32 = 000000100000
  let instr:u32 = u32:0b00000010000010101010101100010011;

  assert_eq(risc_v_imm_gen_2017(instr), u32: 32 );

}


#[test]
fn risc_v_imm_gen_2017_test_lw() {

  // Opcode for lw = 0000011
  // rd = 22 = 10110
  // funct3 = 010
  // rs1 = 21 = 10101
  // immediate = 32 = 000000100000
  let instr:u32 = u32:0b00000010000010101010101100000011;

  assert_eq(risc_v_imm_gen_2017(instr), u32: 32 );

}

#[test]
fn risc_v_imm_gen_2017_test_R_Type() {

  // Opcode for add = 0110011
  // rd = 22 = 10110
  // funct3 = 000
  // rs1 = 21 = 10101
  // rs2 = 22 = 10110
  // funct7 = 0000000
  let instr:u32 = u32:0b00000001011010101000101100110011;

  assert_eq(risc_v_imm_gen_2017(instr), u32: 0 );

}

#[test]
fn risc_v_imm_gen_2017_test_S_Type() {

  // Opcode for sw = 0100011
  // imm[4:0] = 11100
  // funct3 = 010
  // rs1 = 22 = 10110
  // rs2 = 21 = 10101
  // imm[11:7] = 0000000
  let instr:u32 = u32:0b00000011010110110010111000100011;

  assert_eq(risc_v_imm_gen_2017(instr), u32: 60 );

}

#[test]
fn risc_v_imm_gen_2017_test_UJ_Type() {

  // Opcode for jal = 1101111
  // rd = 22 = 10110
  // jump offset is 120 = 0 00000000 0 0000111100 (0) remove for sign extend;
  // Swap the 00000000 and 0000111100
  // Breakdown: 0 0000111100 0 00000000
  let instr:u32 = u32:0b00000111100000000000101101101111;

  assert_eq(risc_v_imm_gen_2017(instr), u32:0b000000000000000001111000 );

}

#[test]
fn risc_v_imm_gen_2017_test_U_Type() {

  // Opcode for lui = 0110111
  // rd = 18 = 10010
  // lui = 248 = 00000000000011111000;
  let instr:u32 = u32:0b00000000000011111000100100110111;

  // Expected Result: 0b00000000000011111000000000000000
  assert_eq(risc_v_imm_gen_2017(instr), u32: 0b00000000000011111000000000000000 );

}

In [ ]:
#@title Run this cell to synthesize the risc_v_imm_gen_2017 cell {display-mode: "form"}
#@markdown - Click the ▷ button to run synthesis, static timing analysis and global placement

placement_density = 1 #@param {type:"slider", min:0, max:1.0, step:0.01}
clock_period_ps = 10000 #@param {type:"slider", min:0, max:100000, step:1}
clock_period_ns = clock_period_ps / 1000.0
core_area = 'absolute' # @param ["relative", "absolute"]

# @markdown ### core_area_relative
# @markdown compute core area from the design size
utilization_percent = 100 #@param {type:"slider", min:0, max:100, step:1}
# @markdown ### core_area_absolute
# @markdown set core area explicitly
core_width_microns = 50 #@param {type:"slider", min:0, max:1000, step:1}
core_padding_microns = 0 #@param {type:"slider", min:0, max:100, step:1}

from IPython.display import display, display_png
import IPython.display
import PIL.Image

if core_area == 'relative':
  core_area_value = RelativeCoreArea(utilization_percent)
else:
  core_area_value = AbsoluteCoreArea(core_width_microns, core_padding_microns)

tb = widgets.TabBar(['synthesis', 'netlist', 'timing', 'placement', 'area', 'power'])

# run yosys synthesis
with tb.output_to('synthesis', select=True):
  synth_results = run_synthesis()
  tb.clear_tab()

with tb.output_to('synthesis', select=False):
  grid = widgets.Grid(1, 2, header_row=False, header_column=False)
  with grid.output_to(0, 0):
    display(synth_results.cell_stats)
  with grid.output_to(0, 1):
    display(synth_results.design_stats)

# display gate level netlist
with tb.output_to('netlist', select=False):
  with synth_results.synth_v.open('r') as f:
    print(f.read())


# run opensta static timing analysis
with tb.output_to('timing', select=True):
  opensta_results = run_opensta()
  tb.clear_tab()

# display opensta report
with tb.output_to('timing', select=False):
  display(
      opensta_results.style.hide(axis='index')
      .background_gradient(subset=['delay'], cmap='Oranges')
      .bar(subset=['time'], color='lightblue')
  )

# run openroad placement
with tb.output_to('placement', select=True):
  placement_results = run_placement(
      clock_period_ps=clock_period_ps,
      placement_density=placement_density,
      core_area=core_area_value,
  )
  tb.clear_tab()

# display global placement layout
with tb.output_to('placement', select=False):
  if placement_results.openroad_global_placement_layout.exists():
    img = PIL.Image.open(placement_results.openroad_global_placement_layout)
    img = img.resize((500, 500))
    display_png(img)

# display area estimate
with tb.output_to('area', select=False):
  display(
      placement_results.area.style.format('{:.3f} μm²', subset=['area'])
      .format('{:.2f} %', subset=['utilization'])
      .bar(subset=['utilization'], color='lightblue', vmin=0, vmax=100)
  )

# display power metrics
with tb.output_to('power', select=False):
  display(
      placement_results.power.style.format('{:.3f} uW')
      .background_gradient(
          subset=pd.IndexSlice[
              placement_results.power.index[:-1], ['internal', 'switching', 'leakage']
          ],
          cmap='Oranges',
          axis=None,
      )
      .bar(subset=['total'], color='lightcoral')
      .bar(
          subset=pd.IndexSlice[placement_results.power.index[-1:], :],
          color='lightcoral',
          axis='columns',
      )
  )

## Results - We Proved the RISC-V Inventors Claim!

In our comparison of results, we obtained the following:

|Metric|RISC-V 2011|RISC-V 2017|Improvement|
|:--|:--|:--|:--|
|time - Under the delay tab| 1.02 | 0.77 | 2017 by 24.51% |
|global placement - under the area tab| 624.349 μm² | 465.446 μm² | 2017 by 25.45% |
|combinational - under the power tab| 0.599 μW | 0.592 μW | 2017 by 1.16% |

We may conclude that the 2017 RISC-V redesign gained an improvement in 24.51% in <b>area</b> and a 25.45% improvement in <b>performance</b> with a slight improvement in power consumption of 1.16%.

> To quote the movie "Oppenheimer" - It's paradoxical, and yet... it works!

## Combining Concepts: DSLX structs, match statements, and the RISC-V 2017 Immediate Generate

Keeping track of all these bits and bytes is challenging, even for the most advanced programmers. So let us revisit the idea of DSLX structs and re-write the <code>risc_v_imm_gen_2017</code> module with a RISC-V instruction struct.

Revisiting the 2017 specification, we can observe some patterns

<img src = "https://github.com/mmorri22/cse30321/blob/main/xls/lec24/updated_immediate.png?raw=true" width=600>

## Controller, ALU Controller, and ALU Design

Next, we will complete the combinational design aspects of the RISC-V datapath by discussing the controller, the ALU control unit, and the ALU design itself.

> Note: The controller for the RISC-V datapath is combinational and updated every cycle based on the opcode. This is different than the control unit in AlbaCore, which uses a Finite State Machine (which we will cover in Reading 14).

Recall from zyBook Participation Activity 4.4.1: How the ALU control bits are set depends on the ALUOp control bits and the different function codes for the R-type instruction.



### Full ALU Controller Truth Table

|Signal|Operation|
|:--|:--|
|<code>0000</code>|AND|
|<code>0001</code>|OR|
|<code>0010</code>|add|
|<code>0110</code>|add|

## Putting the Combinational Pieces Together

Now that we have multiplexers, ALU control, the Control Unit itself, and the Data Forwarding Logic, we can put the pieces together on all the combinational aspects of the RISC-V Pipelined datapath. In the diagram below, we see all the elements that we will design and test.
<ul>
  <li>Control Unit</li>
  <li>ALU Control Unit</li>
  <li>Multiplexers</li>
  <li>Immediate Generator</li>
  <li>Arithmetic Logic Unit</li>
  <li>Hazard Detection and Data Forwarding Unit</li>
</ul>

The image below shows the RISC-V pipelined datapath that we designed in class, and the busses in bold and Notre Dame gold are the parts that are represented in the <code>risc_v_combinational</code> module below

<img src = "
https://github.com/mmorri22/cse30321/blob/main/xls/lec24/combinational_risc_v.png?raw=true" width=800>

In [ ]:
%%dslx --top=riscv_simple_alu --pipeline_stages=1 --flop_inputs=false --flop_outputs=false

const ALU_OP_AND = u4:0;    // 0000
const ALU_OP_OR = u4:1;     // 0001
const ALU_OP_ADD = u4:2;    // 0010
const ALU_OP_XOR = u4:3;    // 0011
const ALU_OP_SUB = u4:6;    // 0110
const ALU_OP_SLT = u4:7;    // 0111
const ALU_OP_SLL = u4:8;    // 0100
const ALU_OP_SRL = u4:9;    // 1001
const ALU_OP_LUI = u4:10;   // 1010
const ALU_OP_NOP = u4:15;  // 1111

fn twos_complement(value: u33, bit:u1) -> u33{

    match(bit){
      u1:0 => value,

      _ => {
        let sign_extend_val:u33 = signex(bit, u33:1);
        (value ^ sign_extend_val) + u33:1
      }
    }
}

fn add_sub_32_with_carry(a: u32, b: u32, add_sub: u1) -> (u1, u32) {

  // Expand both busses from 32 to 33
  let a_with_overflow:u33 = a as u33;
  let b_with_overflow:u33 = twos_complement(b as u33, add_sub);

  let result_with_carry = a_with_overflow + b_with_overflow;

  // Get the first eight bits
  // This notation uses the same notation as range in Python
  // [0:32] actually gets the 32 bits from 0 to 31 where 32 is *exclusive*
  let result = result_with_carry[0:32];

  // To access one bit, use the bit you want and +:u1
  let carry_bit = result_with_carry[32+:u1];

  // Finally, return the sum and carry bit
  (carry_bit, result)
}


// We will design the ALU together here


In [ ]:
#@title Run this cell to synthesize the riscv_simple_alu cell {display-mode: "form"}
#@markdown - Click the ▷ button to run synthesis, static timing analysis and global placement

placement_density = 1 #@param {type:"slider", min:0, max:1.0, step:0.01}
clock_period_ps = 10000 #@param {type:"slider", min:0, max:100000, step:1}
clock_period_ns = clock_period_ps / 1000.0
core_area = 'absolute' # @param ["relative", "absolute"]

# @markdown ### core_area_relative
# @markdown compute core area from the design size
utilization_percent = 100 #@param {type:"slider", min:0, max:100, step:1}
# @markdown ### core_area_absolute
# @markdown set core area explicitly
core_width_microns = 150 #@param {type:"slider", min:0, max:1000, step:1}
core_padding_microns = 0 #@param {type:"slider", min:0, max:100, step:1}

from IPython.display import display, display_png
import IPython.display
import PIL.Image

if core_area == 'relative':
  core_area_value = RelativeCoreArea(utilization_percent)
else:
  core_area_value = AbsoluteCoreArea(core_width_microns, core_padding_microns)

tb = widgets.TabBar(['synthesis', 'netlist', 'timing', 'placement', 'area', 'power'])

# run yosys synthesis
with tb.output_to('synthesis', select=True):
  synth_results = run_synthesis()
  tb.clear_tab()

with tb.output_to('synthesis', select=False):
  grid = widgets.Grid(1, 2, header_row=False, header_column=False)
  with grid.output_to(0, 0):
    display(synth_results.cell_stats)
  with grid.output_to(0, 1):
    display(synth_results.design_stats)

# display gate level netlist
with tb.output_to('netlist', select=False):
  with synth_results.synth_v.open('r') as f:
    print(f.read())


# run opensta static timing analysis
with tb.output_to('timing', select=True):
  opensta_results = run_opensta()
  tb.clear_tab()

# display opensta report
with tb.output_to('timing', select=False):
  display(
      opensta_results.style.hide(axis='index')
      .background_gradient(subset=['delay'], cmap='Oranges')
      .bar(subset=['time'], color='lightblue')
  )

# run openroad placement
with tb.output_to('placement', select=True):
  placement_results = run_placement(
      clock_period_ps=clock_period_ps,
      placement_density=placement_density,
      core_area=core_area_value,
  )
  tb.clear_tab()

# display global placement layout
with tb.output_to('placement', select=False):
  if placement_results.openroad_global_placement_layout.exists():
    img = PIL.Image.open(placement_results.openroad_global_placement_layout)
    img = img.resize((500, 500))
    display_png(img)

# display area estimate
with tb.output_to('area', select=False):
  display(
      placement_results.area.style.format('{:.3f} μm²', subset=['area'])
      .format('{:.2f} %', subset=['utilization'])
      .bar(subset=['utilization'], color='lightblue', vmin=0, vmax=100)
  )

# display power metrics
with tb.output_to('power', select=False):
  display(
      placement_results.power.style.format('{:.3f} uW')
      .background_gradient(
          subset=pd.IndexSlice[
              placement_results.power.index[:-1], ['internal', 'switching', 'leakage']
          ],
          cmap='Oranges',
          axis=None,
      )
      .bar(subset=['total'], color='lightcoral')
      .bar(
          subset=pd.IndexSlice[placement_results.power.index[-1:], :],
          color='lightcoral',
          axis='columns',
      )
  )

# 📄 README

Like what you see? 🤝 [Contact us](https://docs.google.com/forms/d/e/1FAIpQLSd1DNMoOxxr73mkIrZXhDWd1gn-jSsL7SMQry6y_JK0caDKlg/viewform?resourcekey=0-1YtZY34PHo-vug_UmFrMQg) 💬 [Join the chat](https://chat.google.com/room/AAAA8aUpxQc?cls=4)

# 🔒 Privacy

 `%%dslx` cell execution count is tracked using [Google Analytics](https://developers.google.com/analytics).